# Even / Odd denoising of empiar11415

* Inputs: `even.mrc` and `odd.mrc`, the pair of noisy vols.
* Outputs: `even_odd_denoised/empiar11415_stack_crop.mrc` with the denoised vol, and `denoise_even_odd.pdf` with a view of a tile of a central slice in Z.

In [ ]:
from pathlib import Path
from collections import namedtuple
from my_google_auth import DriveHandler

In [ ]:
Args = namedtuple("args", ["even", "odd", "denoised"])
args = Args("even.mrc", "odd.mrc", "even_odd_denoised/empiar11415.mrc")

In [ ]:
if Path(args.denoised).exists():
    raise Exception(f"{args.denoised} already exists ... exiting")

In [ ]:
if not Path("even.mrc").exists() or not Path("odd.mrc"):
    %run split_even_odd.ipynb

In [ ]:
from my_google_auth import DriveHandler
service = DriveHandler.get_drive_service()
handler = DriveHandler.DriveHandler(service)
MY_SHARED_DRIVE_ID = "1hGHvkP46fxLCQbUlyYhAS_eVl6PollQM"

In [ ]:
# %pip install tensorflow==2.4

## Configure cryoCARE

In [ ]:
%%writefile train_data_config__evenodd.json
{
    "even": ["even.mrc"],
    "odd": ["odd.mrc"],
    "mask": [""],
    "patch_shape": [16, 16, 16],
    "num_slices": 360,
    "split": 0.9,
    "tilt_axis": "Y",
    "n_normalization_samples": 200,
    "path": "./even_odd_data",
    "overwrite": "True"  
}

In [ ]:
%%bash
#cd /nas/vruiz/cryoCARE/empiar11415
source ~/envs/cryoCARE/bin/activate
cryoCARE_extract_train_data.py --conf train_data_config__evenodd.json

## Train

In [ ]:
%%writefile train_config__evenodd.json
{
  "train_data": "./even_odd_data",
  "epochs": 50,
  "steps_per_epoch": 200,
  "batch_size": 16,
  "unet_kern_size": 3,
  "unet_n_depth": 3,
  "unet_n_first": 16,
  "learning_rate": 0.0004,
  "model_name": "model",
  "path": "./",
  "gpu_id": [1]
}

In [ ]:
%%bash
#cd /nas/vruiz/cryoCARE/empiar11415
source ~/envs/cryoCARE/bin/activate
cryoCARE_train.py --conf train_config__evenodd.json

In [ ]:
uploaded_file_id = handler.upload(
    local_file_path='model.tar.gz',
    drive_file_name='empiar11415__denoised__cryoCARE_even_odd__model.tar.gz',
    drive_folder_id=MY_SHARED_DRIVE_ID
)

## Infer

In [ ]:
%%bash
ln -sf /nas/vruiz/vols/empiar11415.mrc .

In [ ]:
%%writefile predict_config__evenodd.json
{
    "path": "./model.tar.gz",
    "even": ["empiar11415.mrc"], 
    "odd": ["empiar11415.mrc"],
    "n_tiles": [1,2,2],
    "output": "even_odd_denoised",
    "overwrite": "True",
    "gpu_id": [1]
}

In [ ]:
\subsection{cryoCARE-EO (cryoCARE Even Odd)}
%%bash
#cd /nas/vruiz/cryoCARE/empiar11415
pwd
source ~/envs/cryoCARE/bin/activate
cryoCARE_predict.py --conf predict_config__evenodd.json || true

In [ ]:
uploaded_file_id = handler.upload(
    local_file_path='even_odd_denoised/empiar11415.mrc',
    drive_file_name='empiar11415__denoised__cryoCARE_even_odd.mrc',
    drive_folder_id=MY_SHARED_DRIVE_ID
)

In [ ]:
import mrcfile
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [ ]:
def read_MRC(path_file):
    return mrcfile.read(mrc_file_path)

In [ ]:
mrc_file_path = '/nas/vruiz/vols/empiar11415.mrc'
original_volume = read_MRC(mrc_file_path)

In [ ]:
original_volume.shape

In [ ]:
mrc_file_path = 'even_odd_denoised/empiar11415.mrc'
denoised_volume = read_MRC(mrc_file_path)

In [ ]:
denoised_volume.shape

In [ ]:
# Choose a slice index in the middle of the volume for a good comparison
slice_idx = original_volume.shape[0] // 2

fig, axes = plt.subplots(1, 2, figsize=(20, 20))

# Plot the original slice z
im1 = axes[0].imshow(original_volume[slice_idx, :, :].T, cmap='gray', origin='lower')
axes[0].set_title(f'Original Slice Z={slice_idx}')
axes[0].grid(False)

# Plot the original slice z+1
im2 = axes[1].imshow(denoised_volume[slice_idx, :, :].T, cmap='gray', origin='lower')
axes[1].set_title(f'N2N Even/Odd Denoised Slice Z={slice_idx}')
axes[1].grid(False)

plt.tight_layout()
plt.show()

In [ ]:
# Choose a slice index in the middle of the volume for a good comparison
slice_idx = original_volume.shape[0] // 2

fig, axes = plt.subplots(1, 2, figsize=(20, 20))

# Plot the original slice z
im1 = axes[0].imshow(original_volume[slice_idx, 0:500, 0:500].T, cmap='gray', origin='lower')
axes[0].set_title(f'Original Slice Z={slice_idx}')
axes[0].grid(False)

# Plot the original slice z+1
im2 = axes[1].imshow(denoised_volume[slice_idx, 0:500, 0:500].T, cmap='gray', origin='lower')
axes[1].set_title(f'N2N Even/Odd Denoised Slice Z={slice_idx}')
axes[1].grid(False)

plt.tight_layout()
plt.show()

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(16, 16))
slice_idx = denoised_volume.shape[0]//2
plt.imshow(denoised_volume[slice_idx, 0:500, 0:500], cmap="gray")
plt.savefig("denoise_even_odd.pdf")

In [ ]:
uploaded_file_id = handler.upload(
    local_file_path='denoise_even_odd.pdf',
    drive_file_name='empiar11415__denoised__cryoCARE_even_odd.pdf',
    drive_folder_id=MY_SHARED_DRIVE_ID
)

In [ ]:
%pip install "self_fourier_shell_correlation @ git+https://github.com/vicente-gonzalez-ruiz/self_fourier_shell_correlation"

In [ ]:
%pip show self_fourier_shell_correlation

In [ ]:
import sys
print(sys.executable)

In [ ]:
%pip install "shuffling @ git+https://github.com/vicente-gonzalez-ruiz/shuffling"

In [ ]:
%pip install opencv-python

In [ ]:
%pip install "motion_estimation @ git+https://github.com/vicente-gonzalez-ruiz/motion_estimation"

In [ ]:
from self_fourier_shell_correlation import fsc_utils as fsc

In [ ]:
import mrcfile

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
even = mrcfile.read("even.mrc")

In [ ]:
odd = mrcfile.read("odd.mrc")

In [ ]:
spatial_freq, fsc_values = fsc.fourier_shell_correlation(even, odd, shell_thickness=0.01)

In [ ]:
fsc.plot_fsc(spatial_freq, fsc_values, "Spatial Frequency (1/voxel units)", "FSC", "FSC(even, odd)")